# Lösningsförslag till Övningsuppgift - Stopped Flow
Författare: Björn Dahlgren, 2016-04-04

Se övningsuppgiften för detaljer. Kortfattat ska vi bestämma hastighetskonstanten för följande reaktion:

$$
3A + B \rightarrow C
$$

Detta lösningsförslag är skrivet i Python, se bilaga till laborationshandledningen för liknande exempel skrivna för Matlab.

In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.linalg import lstsq
%matplotlib inline

Från uppgiftstexten vet vi att vi har data för 5 värden på [B]₀, 10 replikat för vardera, och 42 tidpunkter per mätserie. Vi läser in filerna in i variabeln ``data``:

In [ ]:
A0 = 1e-3
n_conc, n_rep, n_t = 5, 10, 42
conc_B = np.linspace(1/10, n_conc/10, n_conc)  # molar
data = np.empty((n_conc, n_rep, n_t, 2))
for i in range(n_conc):
    for j in range(n_rep):
        data[i, j, :, :] = np.loadtxt('data/0.%d/%d.dat' % (i+1, j+1))

Om vi antar pseudo tredje ordningens reaktion ([B]₀ » [A]₀) blir hastighetsuttrycket för reaktionen:

$$
\frac{dA}{dt} = -3kA^3
$$

integrering ger:
$$
A(t) = \frac{1}{\sqrt{\frac{1}{A_0^2} + 6kt}}
$$

genom Lambert-Beers lag får vi absorbansen ($y$):

$$
y(t) = \frac{\varepsilon l}{\sqrt{\frac{1}{A_0^2} + 6kt}}
$$

utifrån detta uttryck skapar vi en funktion för som vi kallar "``model``":

In [ ]:
def model(t, eps_l, k):
    return eps_l*(A0**-2 + 6*k*t)**-0.5

## Ickelinjär kurvanpassning
Vi passar vår modellekvation på var och en av dataserierna. Vi formulerar även en relativ vikt baserad på hur väl passningen är. Valet av vikt är något godtyckligt och det kan vara bra att prova sig fram, ett vanligt val är Mean Square Deviation (MSD) & Mean Absolute Deviation (MAD). Vi använder MSD nedan:

In [ ]:
popt = np.empty((n_conc, n_rep, 2))  # epsilon*l, k
pcov = np.empty((n_conc, n_rep, 2, 2))
all_k = np.empty((n_conc, n_rep))
all_w = np.empty((n_conc, n_rep))
all_m = np.empty((n_conc, n_rep, n_t))
plt.figure(figsize=(16, 4))
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    for j in range(n_rep):
        t = data[i, j, :, 0]
        y = data[i, j, :, 1]
        try:
            popt[i, j, :], pcov[i, j, :, :] = curve_fit(model, t, y, [y[0]/A0, 1])
        except:
            print('failed')
            popt[i, j, :], pcov[i, j, :, :] = [0]*2, [[float('inf')]*2]*2
        plt.plot(t, y, 'k', alpha=0.4)
        all_k[i, j] = popt[i, j, 1]
        all_m[i, j, :] = m = model(t, *popt[i, j, :])
        # Val av vikt:
        #all_w[i, j] = 1/pcov[i, j, 0, 0]             # varians i epsilon*l
        #all_w[i, j] = 1/pcov[i, j, 1, 1]             # varians i k
        #all_w[i, j] = 1/np.average(np.square(y-m))    # MSD
        all_w[i, j] = 1/(np.average(np.abs(y-m)))    # MAD

for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
    ax.autoscale(tight=True)
    ax.set_ylabel('Absorbance')
    ax.set_xlabel('Time / s')
    for j in range(n_rep):
        red = 1 - all_w[i, j]/np.max(all_w[i, :])
        t = data[i, j, :, 0]
        plt.plot(t, all_m[i, j, :], c=(red, 0, 1.0))
plt.tight_layout()

## Poolning av regressioner
Nu beräknar vi de viktade medelvärdena (se laborationshandledning för formel) för de olika värdena för [B]₀ baserat på respektive 10 replikat, samt respektive viktade stickprovsvarians (se laborationshandledning för formel).

In [ ]:
def weighted_average(obs, s2):
    avg, sum_of_w = np.average(obs, axis=0, weights=1/s2, returned=True)
    var = np.sum(np.square(obs - avg)/s2, axis=0)/((avg.shape[0] - 1) * sum_of_w)
    return avg, var

In [ ]:
def weighted_average_plot(obs, s2, xlbl=r'$\beta_0$', ylbl=r'$\beta_1$', ttl=r'$y(x) = \beta_0 + \beta_1 \cdot x$',
                         label_cb=None):
    avg, var = weighted_average(obs, s2)
    lbl = None if label_cb is None else label_cb(avg, var)
    plt.errorbar(avg[0], avg[1], xerr=var[0]**0.5, yerr=var[1]**0.5, marker='o', c='r',
                 linewidth=2, markersize=10, label=lbl)
    plt.errorbar(obs[:, 0], obs[:, 1], marker='s', ls='None', xerr=s2[:, 0]**0.5, yerr=s2[:, 1]**0.5, alpha=.5)
    plt.xlabel(xlbl); plt.ylabel(ylbl); plt.title(ttl)
    return avg, var

In [ ]:
plt.figure(figsize=(16, 4))
beta_conc = np.empty((n_conc, 2))
beta_conc_var = np.empty_like(beta_conc)
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    beta_conc[i, :], beta_conc_var[i, :] = weighted_average_plot(popt[i, ...], pcov[i, :, [0, 1], [0, 1]].T)
plt.tight_layout()
beta_conc[:, 1], beta_conc_var[:, 1]

Våra pseudo tredje ordningens hastighetskonstanter är linjärt beroende av [B]₀:

$$
k_f = [B]_0 k
$$

Därför beräknar vi $k_f$ genom [viktad linjär regression](https://en.wikipedia.org/wiki/Linear_least_squares_%28mathematics%29#Weighted_linear_least_squares):

In [ ]:
def LS(x, y, w=1):  # w == 1 => OLS, w != 1 => WLS
    """ Least squares 
    
    References
    -----------
    Wikipedia & standard texts on least squares.
    A note about R2 in WLS:
        Willett, John B., and Judith D. Singer. "Another cautionary note about R 2:
        Its use in weighted least-squares regression analysis."
        The American Statistician 42.3 (1988): 236-238.
    """
    sqrtw = np.sqrt(w)
    y = y * sqrtw
    X = np.ones((x.size, 2))
    X[:, 1] = x
    if hasattr(sqrtw, 'ndim') and sqrtw.ndim == 1:
        sqrtw = sqrtw.reshape((sqrtw.size, 1))
    X *= sqrtw
    
    beta = np.linalg.lstsq(X, y)[0]
    eps = X.dot(beta) - y
    SSR = eps.T.dot(eps)  # sum of squared residuals
    vcv = SSR/(x.size - 2)*np.linalg.inv(X.T.dot(X))
    TSS = np.sum(np.square(y - np.mean(y)))  # total sum of squares
    R2 = 1 - SSR/TSS
    return beta, vcv, R2
    XtX = X.T.dot(X)
    return np.linalg.lstsq(XtX, X.T.dot(y))[0]

In [ ]:
beta, M_beta, R2 = LS(conc_B, beta_conc[:, 1], 1/beta_conc_var[:, 1])
beta, M_beta

In [ ]:
plt.errorbar(conc_B, beta_conc[:, 1], 3*np.sqrt(beta_conc_var[:, 1]), c='k', ls='None', marker='.')
lbl = '$k_f = (%.2f \pm %.2g) M^{-3}s^{-1}$' % (beta[1], M_beta[1, 1]**0.5)
plt.plot(conc_B, beta[0] + beta[1]*conc_B, 'b', label=lbl)
plt.gca().set_xlim((0, conc_B[-1]*1.05))
plt.gca().set_ylim((0, (beta[1]*conc_B)[-1]*1.1))
_ = plt.legend(loc='best')

Och därmed är uppgiften löst. Värdet på $k_f$ känns bekannt (eftersom våra indata är syntetiska stod det mig fritt att välja det "sanna" $k_f$).

# Minsta-kvadrat-anpassning till det linjäriserade problemet
Som ett alternativ till icke-linjär ekvationsanpassning skulle vi kunna ha linjäriserat vårt uttryck för absorbansen:

$$
\frac{1}{y^2(t)} = \frac{\frac{1}{A_0^2} + 6kt}{\varepsilon^2 l^2}
$$

därifrån kan vi lösa ut vårt pseudo tredje ordningens hastighetskonstant:

$$
\frac{1}{y^2(t)} = \frac{1}{\varepsilon^2 l^2 A_0^2} + \frac{6k}{\varepsilon^2 l^2}t \\
$$

notera att:

$$
\frac{1}{y^2(t)} \propto t
$$

vi inför därför en linjär funktion f(t) för regression:

$$
f(t) = p_0 + p_1 t
$$

$$
\begin{cases}
p_0 = \frac{1}{\varepsilon^2 l^2 A_0^2} \\
p_1 = \frac{6k}{\varepsilon^2 l^2}
\end{cases}
$$

$$
\begin{cases}
\varepsilon^2 l^2 = \frac{1}{p_0 A_0^2} \\
p_1 = \frac{6k}{\varepsilon^2 l^2}
\end{cases}
$$

$$
k = \frac{p_1}{6 p_0 A_0^2}
$$


In [ ]:
all_k = np.zeros((n_conc, n_rep))
var_k = np.empty_like(all_k)
plt.figure(figsize=(14, 4))
for i in range(n_conc):
    for j in range(n_rep):
        t = data[i, j, :, 0]
        f = data[i, j, :, 1]**-2
        beta, cov, R2 = LS(t, f)
        all_k[i, j] = beta[1]/(6*beta[0]*1e-6)
        var_k[i, j] = 1e6/6*(cov[0, 0] + cov[1, 1])
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
    max_var_k = np.max(var_k[i, :])
    for j in range(n_rep):
        t = data[i, j, :, 0]
        f = data[i, j, :, 1]**-2
        plt.plot(t, f, c=(var_k[i, j]/max_var_k, 0, 0), alpha=0.3)  # Ju rödare, desto större osäkerhet i k

In [ ]:
k_conc, k_conc_s2 = weighted_average(all_k.T, 1/var_k.T)
beta, beta_cov, R2 = LS(conc_B, k_conc, k_conc_s2)
plt.errorbar(conc_B, k_conc, yerr=np.sqrt(k_conc_s2), label='k = %.2f +/- %.2f' % (beta[1], beta_cov[1, 1]**0.5),
             marker='s', ls='None')
plt.plot(conc_B, beta[0] + conc_B*beta[1])
_ = plt.legend(numpoints=1, loc='best')

Som förväntat ger inte denna metod ett lika tillförlitligt resultat.